In [1]:
import asyncio
import websockets
import json
import logging
from jetbot import Robot, Camera, bgr8_to_jpeg
import base64

# Logging Setup
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Configuration
WEBSOCKET_PORT = 8766
CAMERA_WIDTH = 300
CAMERA_HEIGHT = 300
FPS = 10

# JetBot Initialization
try:
    robot = Robot()
    camera = Camera.instance(width=CAMERA_WIDTH, height=CAMERA_HEIGHT)
    logger.info("JetBot initialized successfully.")
except Exception as e:
    logger.error(f"JetBot initialization failed: {e}")
    robot = None
    camera = None

# Command Handling
async def handle_command(command):
    if not robot:
        logger.error("Robot not initialized.")
        return
    try:
        speeds = {
            "forward_slow": (0.2, 0.2), "forward_medium": (0.4, 0.4), "forward_fast": (0.6, 0.6),
            "backward_slow": (-0.2, -0.2), "backward_medium": (-0.4, -0.4), "backward_fast": (-0.6, -0.6),
            "left_slow": (-0.1, 0.2), "left_medium": (-0.2, 0.4), "left_fast": (-0.3, 0.6),
            "right_slow": (0.2, -0.1), "right_medium": (0.4, -0.2), "right_fast": (0.6, -0.3),
            "stop": (0.0, 0.0)
        }
        left_speed, right_speed = speeds.get(command, (0.0, 0.0))
        robot.set_motors(left_speed, right_speed)
        logger.info(f"Command: {command}, Motors: Left={left_speed}, Right={right_speed}")
    except Exception as e:
        logger.error(f"Command execution error: {e}")
        robot.stop()

# WebSocket Handler
async def websocket_handler(websocket, path):
    logger.info("WebSocket connection established")
    async def send_image_stream():
        if not camera:
            return
        while True:
            try:
                frame = camera.value
                if frame is not None:
                    image_base64 = base64.b64encode(bgr8_to_jpeg(frame)).decode('utf-8')
                    await websocket.send(json.dumps({"image": image_base64}))
                await asyncio.sleep(1.0 / FPS)
            except Exception as e:
                logger.error(f"Image stream error: {e}")
                break

    image_task = asyncio.ensure_future(send_image_stream())
    try:
        async for message in websocket:
            try:
                data = json.loads(message)
                command = data.get("command", "stop")
                await handle_command(command)
            except json.JSONDecodeError:
                logger.error("Invalid JSON received")
            except Exception as e:
                logger.error(f"Message handling error: {e}")
    except websockets.ConnectionClosed:
        logger.info("WebSocket connection closed by client")
    except Exception as e:
        logger.error(f"WebSocket error: {e}")
    finally:
        image_task.cancel()
        if robot:
            robot.stop()
            logger.info("Motors stopped on disconnect")

# Main Function
async def main():
    while True:
        try:
            server = await websockets.serve(websocket_handler, "0.0.0.0", WEBSOCKET_PORT)
            logger.info(f"WebSocket server running on port {WEBSOCKET_PORT}")
            await asyncio.Future()
        except Exception as e:
            logger.error(f"Server error: {e}")
            await asyncio.sleep(5)
            logger.info("Retrying server start...")

if __name__ == "__main__":
    if robot:
        robot.stop()
    loop = asyncio.get_event_loop()
    try:
        loop.run_until_complete(main())
        loop.run_forever()
    except KeyboardInterrupt:
        logger.info("Shutting down server")
        if robot:
            robot.stop()
    finally:
        loop.close()

2025-03-26 08:50:17,575 - ERROR - JetBot initialization failed: Could not initialize camera.  Please see error trace.


RuntimeError: Cannot close a running event loop

2025-03-26 08:50:17,987 - INFO - WebSocket server running on port 8766
2025-03-26 08:50:25,143 - INFO - WebSocket connection established
2025-03-26 08:50:51,568 - INFO - WebSocket connection established


In [ ]:
import asyncio
import websockets
import json
import logging
from jetbot import Robot, Camera, bgr8_to_jpeg
import base64

# Logging Setup
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Configuration
WEBSOCKET_PORT = 8766
CAMERA_WIDTH = 300
CAMERA_HEIGHT = 300
FPS = 10

# JetBot Initialization
try:
    robot = Robot()
    camera = Camera.instance(width=CAMERA_WIDTH, height=CAMERA_HEIGHT)
    logger.info("JetBot initialized successfully.")
except Exception as e:
    logger.error(f"JetBot initialization failed: {e}")
    robot = None
    camera = None

# Command Handling
async def handle_command(command):
    if not robot:
        logger.error("Robot not initialized.")
        return
    try:
        speeds = {
            "forward_slow": (0.2, 0.2), "forward_medium": (0.4, 0.4), "forward_fast": (0.6, 0.6),
            "backward_slow": (-0.2, -0.2), "backward_medium": (-0.4, -0.4), "backward_fast": (-0.6, -0.6),
            "left_slow": (-0.1, 0.2), "left_medium": (-0.2, 0.4), "left_fast": (-0.3, 0.6),
            "right_slow": (0.2, -0.1), "right_medium": (0.4, -0.2), "right_fast": (0.6, -0.3),
            "stop": (0.0, 0.0)
        }
        left_speed, right_speed = speeds.get(command, (0.0, 0.0))
        robot.set_motors(left_speed, right_speed)
        logger.info(f"Command: {command}, Motors: Left={left_speed}, Right={right_speed}")
    except Exception as e:
        logger.error(f"Command execution error: {e}")
        robot.stop()

# WebSocket Handler
async def websocket_handler(websocket, path):
    logger.info("WebSocket connection established")
    async def send_image_stream():
        if not camera:
            return
        while True:
            try:
                frame = camera.value
                if frame is not None:
                    image_base64 = base64.b64encode(bgr8_to_jpeg(frame)).decode('utf-8')
                    await websocket.send(json.dumps({"image": image_base64}))
                await asyncio.sleep(1.0 / FPS)
            except Exception as e:
                logger.error(f"Image stream error: {e}")
                break

    image_task = asyncio.ensure_future(send_image_stream())
    try:
        async for message in websocket:
            try:
                data = json.loads(message)
                command = data.get("command", "stop")
                await handle_command(command)
            except json.JSONDecodeError:
                logger.error("Invalid JSON received")
            except Exception as e:
                logger.error(f"Message handling error: {e}")
    except websockets.ConnectionClosed:
        logger.info("WebSocket connection closed by client")
    except Exception as e:
        logger.error(f"WebSocket error: {e}")
    finally:
        image_task.cancel()
        if robot:
            robot.stop()
            logger.info("Motors stopped on disconnect")

# Main Function
async def main():
    while True:
        try:
            server = await websockets.serve(websocket_handler, "0.0.0.0", WEBSOCKET_PORT)
            logger.info(f"WebSocket server running on port {WEBSOCKET_PORT}")
            await asyncio.Future()
        except Exception as e:
            logger.error(f"Server error: {e}")
            await asyncio.sleep(5)
            logger.info("Retrying server start...")

if __name__ == "__main__":
    if robot:
        robot.stop()
    loop = asyncio.get_event_loop()
    try:
        loop.run_until_complete(main())
        loop.run_forever()
    except KeyboardInterrupt:
        logger.info("Shutting down server")
        if robot:
            robot.stop()
    finally:
        loop.close()

2025-03-26 07:50:17,015 - INFO - WebSocket server running on port 8766
2025-03-26 07:51:16,057 - INFO - WebSocket connection established
2025-03-26 07:52:13,195 - INFO - Command: forward_slow, Motors: Left=0.2, Right=0.2
2025-03-26 07:52:16,296 - INFO - Command: forward_slow, Motors: Left=0.2, Right=0.2
2025-03-26 07:52:18,828 - INFO - Command: forward_slow, Motors: Left=0.2, Right=0.2
2025-03-26 07:52:21,393 - INFO - Command: forward_slow, Motors: Left=0.2, Right=0.2
2025-03-26 07:52:23,948 - INFO - Command: forward_slow, Motors: Left=0.2, Right=0.2
2025-03-26 07:52:26,505 - INFO - Command: forward_slow, Motors: Left=0.2, Right=0.2
2025-03-26 07:52:29,019 - INFO - Command: forward_slow, Motors: Left=0.2, Right=0.2
2025-03-26 07:52:31,583 - INFO - Command: forward_slow, Motors: Left=0.2, Right=0.2
2025-03-26 07:52:34,164 - INFO - Command: forward_slow, Motors: Left=0.2, Right=0.2
2025-03-26 07:52:36,695 - INFO - Command: forward_slow, Motors: Left=0.2, Right=0.2
2025-03-26 07:52:39,246